# Setup

In [ ]:
# Docker and Milvus Standalone

# 1. wget https://github.com/milvus-io/milvus/releases/download/v2.0.2/milvus-standalone-docker-compose.yml -O docker-compose.yml
# 2. https://docs.docker.com/compose/install/
# 3. Start: sudo docker-compose up -d 
#     Check:sudo docker-compose ps
#     Stop: sudo docker-compose down
#     Delete data after stop: sudo rm -rf  volumes


In [2]:
# Python 3.7.1 or later is required

In [ ]:
# pymilvus

# python3 -m pip install pymilvus==2.0.2
#  Verify: python3 -c "from pymilvus import Collection"

# Import Libraries

In [ ]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

# Connect to Server

In [ ]:
connections.connect("default", host="localhost", port="19530")

# Create Collection

In [ ]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
hello_milvus = Collection("hello_milvus", schema)

In [ ]:
import random
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]
insert_result = hello_milvus.insert(entities)

# Build Index on Entities

In [ ]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)

# Load to Memory and Vector Similarity Search

In [ ]:
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "l2",
    "params": {"nprobe": 10},
}

## Similartiy Search

In [ ]:
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])

# Result 

In [ ]:
# python3 hello_milvus.py